In [2]:
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.errors as errors
import datetime
import os
import math

from collections import defaultdict

################################################################################
# Helper class
################################################################################

class CosmosDBGuard(cosmos_client.CosmosClient):
    def __init__(self, obj):
        self.obj = obj

    def __enter__(self):
        return self.obj

    def __exit__(self, exception_type, exception_val, trace):
        # extra cleanup in here
        self.obj = None

jobs = None
pipeline_runs = None
helix_submissions = None
helix_workitems = None

with CosmosDBGuard(cosmos_client.CosmosClient("https://coreclr-infra.documents.azure.com:443/", {'masterKey': os.environ["coreclrInfraKey"]} )) as client:
        try:
            helix_workitems_link = "dbs/coreclr-infra/colls/helix-workitems"
            helix_submission_link = "dbs/coreclr-infra/colls/helix-submissions"

            pipeline_link = "dbs/coreclr-infra/colls/runtime-pipelines"
            jobs_link = "dbs/coreclr-infra/colls/runtime-jobs"

            # Get only jobs from 8 may to 11 may
            jobs = list(client.QueryItems(jobs_link,
                                                {
                                                    'query': 'SELECT * FROM root job WHERE job.DateStart>@min_start',
                                                    'parameters': [
                                                        {'name': '@min_start', 'value': "2020-05-08T02:30:52.635-07:00"}
                                                    ]
                                                },
                                                {'enableCrossPartitionQuery': True}))

            pipeline_runs = list(client.ReadItems(pipeline_link, {'maxItemCount':1000}))
            #jobs = list(client.ReadItems(jobs_link, {'maxItemCount':1000}))
            helix_submissions = list(client.ReadItems(helix_submission_link, {'maxItemCount':1000}))
            helix_workitems = list(client.ReadItems(helix_workitems_link, {'maxItemCount':1000}))

            print('Found {0} helix submissions'.format(len(helix_submissions)))
            print('Found {0} helix workitems'.format(len(helix_workitems)))

        except errors.HTTPFailure as e:
            print('Error. {0}'.format(e))

        print()



NameError: name 'client' is not defined